#### BeautifulSoup类的基本元素

|基本元素|说明|
|--|--|
|Tag|标签，最基本的信息组织单元，分别用<>和\</>标明开头和结尾|
|Name|标签的名字，\<p>…\</p>的名字是'p'，格式：<tag>.name|
|Attributes|标签的属性，字典形式组织，格式：<tag>.attrs|
|NavigableString|标签内非属性字符串，\<>...\</>中字符串，格式：<tag>.string|
|Comment|标签内字符串的注释部分，一种特殊的Comment类型|

![](tag.jpg)

**Tag的Name和Attributes：**

如何获取标签，标签名称和标签属性？

In [ ]:
from bs4 import BeautifulSoup

demo = open('html.html','r',encoding='utf-8').read()    # 打开本地的html文件（注意html文件所在的位置）

soup = BeautifulSoup(demo, 'lxml')


任何存在于HTML语法中的标签都可以用 **soup.\<tag>** 访问获得，当HTML文档中存在多个相同\<tag>对应内容时，soup.\<tag>返回第一个：

In [ ]:
soup.title

In [ ]:
print(type(soup.title))

In [ ]:
soup.a

每个\<tag>都有自己的名字，通过\<tag>.name获取，返回字符串类型：

In [ ]:
# 获取title标签的名称
soup.title.name 

In [ ]:
# 获取a标签父标签的名称
soup.a.parent.name

一个\<tag>可以有0或多个属性，通过\<tag>.attrs获取，返回字典类型：

In [ ]:
# 获取a标签的属性，返回字典
soup.a.attrs

In [ ]:
# 单独获取 a 标签 href 属性的属性值
soup.a['href']

In [ ]:
# 利用 get 方法，传入属性的名称，二者等价
soup.a.get('href')

**Tag的NavigableString：**

如何获取标签内部的文字？使用 .string或 .text，例如

In [ ]:
soup.title.string

In [ ]:
soup.a.string

In [ ]:
soup.a.text

注意两者的区别：  
    .string获取目标路径下第一个非标签字符串，返回字符串  
    .text获取目标路径下的子孙非标签字符串，返回字符串

In [ ]:
soup.a.parent.string
type(soup.a.parent.string)

soup.a.parent.text
type(soup.a.parent.text)

**批量提取某标签下所有子孙节点标签的NavigableString**

若想获得多个节点内容，亦可使用.strings和.stripped_strings方法：  
    .strings获取目标路径下所有的子孙非标签字符串，返回生成器  
    .stripped_strings获取目标路径下所有的子孙非标签字符串，**会自动去掉空白字符串**，返回生成器

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open('html.html','r',encoding='utf-8'), 'lxml')
soup

In [ ]:
soup.a.parent.parent

In [ ]:
soup.ul.text

In [ ]:
for t in soup.ul.strings:
    print(repr(t))    # repr() 函数将对象转化为供解释器读取的形式

In [ ]:
for t in soup.ul.stripped_strings:
    print(t)

#### 信息提取的方法

一：完整解析信息的标记形式，再提取关键信息
- 需要标记解析器，例如：bs4库的标签树遍历
- 优点：信息解析准确
- 缺点：提取过程繁琐，速度慢

二：无视标记形式，直接搜索关键信息
- 对信息的文本查找函数即可
- 优点：提取过程简洁，速度较快
- 缺点：提取结果准确性与信息内容相关

#### 基于bs4库的html内容遍历方法

![](tree.jpg)


**上行遍历**
+ .parent：返回当前节点的父节点标签
+ .parents：返回当前节点所有先辈节点标签的迭代类型，仅用于循环遍历

In [ ]:
soup.a.parent

In [ ]:
for adult in soup.a.parents:
    print(adult.name)

**下行遍历**
+ .contents：返回子节点标签的列表
+ .children：返回子节点标签的迭代类型，仅用于循环遍历
+ .descendants：返回包含所有子孙节点标签的迭代类型，仅用于循环遍历

In [ ]:
type(soup.body.contents)
soup.body.contents

In [ ]:
type(soup.body.children)
for child in soup.body.children:
    print(child)

In [ ]:
type(soup.body.descendants)

**平行遍历（按照HTML文本顺序）**
+ .next_sibling：返回下一个平行节点标签
+ .previous_sibling：返回上一个平行节点标签
+ .next_siblings：返回后续所有平行节点标签的迭代类型
+ .previous_siblings：返回前序所有平行节点标签的迭代类型

Notice：实际html标签树的平行标签通常是空白/回车/tab等，因此通常仅使用.next_siblings 和 .previous_siblings循环输出当前节点的兄弟节点。

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.baidu.com/')
res.encoding='utf-8'
soup = BeautifulSoup(res.text,'lxml')

for sibling in soup.a.next_siblings:
    print(repr(sibling))

**Exercise：**

    请大家提取出百度主页的html页面的全部标签名称。返回的结果为列表，不包含None并去除重复元素。

#### 正则表达式(regular expression, regex, RE)

**正则表达式**是一种通用的字符串表达框架，可以用来简洁地表达一组字符串，也可以用来判断某字符串的特征归属。

正则表达式在文本处理中十分常用：
- 表达文本类型的特征
- 查找或替换一组字符串
- 匹配字符串的全部或部分

**正则表达式的常用操作符**

|操作符|说明|实例|
|--|--|--|
|.    |匹配除换行符以外的任意字符||
|[  ] |字符集，对单个字符给出取值范围      |[abc]表示a、b、c，[a‐z]表示a到z单个字符|
|[^ ] |非字符集，对单个字符给出排除范围    |[^abc]表示非a或b或c的单个字符|
|\*   |前一个字符0次或无限次扩展          |abc* 表示 ab、abc、abcc、abccc等|
|+    |前一个字符1次或无限次扩展          |abc+ 表示 abc、abcc、abccc等|
|?    |前一个字符0次或1次扩展             |abc? 表示 ab、abc|
|\|   |左右表达式任意一个                 |abc\|def 表示 abc、def|
|^    |匹配字符串开头                     |^abc表示以abc开头的字符串|
|\$    |匹配字符串结尾                     |abc$表示以abc结尾的字符串|
|(  ) |标记一个子表达式的开始和结束位置。   ||
|\d   |匹配数字，等价于[0‐9]|             |
|\w   |匹配字母或数字或下划线，等价于[A‐Za‐z0‐9_]||
|{m}  |扩展前一个字符m次                  |ab{2}c表示abbc|
|{m,} |扩展前一个字符至少m次               |ab{2,}c表示abbc、abbbc、abbbbc等|
|{m,n} |扩展前一个字符m至n次（含n)         |ab{1,2}c表示abc、abbc|



**re库的使用**

Re库是Python的标准库，主要用于字符串匹配。调用方式：`import re`

`regex = re.compile(pattern, flags=0)`将正则表达式的字符串形式编译成正则表达式对象
```
pattern : 正则表达式的字符串或原生字符串表示
flags : 正则表达式使用时的控制标记
```



#### 基于bs4库的html内容查找方法

`<>.find_all(name, attrs, recursive, string, **kwargs)`  [官方文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#find-all)

*返回*一个列表类型，存储查找的结果  
```
name: 查找所有名字为 name 的标签，name 参数的值可以是字符串，正则表达式，列表或是True。
attrs: 对标签属性值进行检索
recursive: 是否对子孙节点全部检索，默认True，若设为False则仅检索子节点
string: 对<>…</>中字符串区域的检索
```

**检索标签名称**

- 传入字符串：查找与字符串完整匹配的内容

In [ ]:
soup.find_all('a')

- 传入列表：返回与列表中任一元素匹配的内容

In [ ]:
soup.find_all(['a','p'])

- 传入True：匹配任何值

In [ ]:
for tag in soup.find_all(True):
    print(tag.name)

- 传入正则表达式

In [ ]:
import re
for tag in soup.find_all(re.compile('t')):  #### 返回所有包含t的标签名
    print(tag.name)

**检索标签属性**

In [ ]:
soup.find_all(attrs={'target':'_blank'})

In [ ]:
soup.find_all(target='_blank')

In [ ]:
soup.find_all(href=re.compile("page"))

In [ ]:
soup.find_all(href=True)

**string参数**

通过 string 参数可以搜索文档中的字符串内容。string 参数接受字符串、正则表达式、列表、True。

In [ ]:
soup.find_all(string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")])

In [ ]:
soup.find_all('a',string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")])

**limit参数**

find_all()方法返回全部的搜索结构，如果文档树很大那么搜索会很慢。如果我们不需要全部结果，可以使用 limit 参数限制返回结果的数量。

In [ ]:
soup.find_all(string=[re.compile("利好"), re.compile("新高"), re.compile("扩大")], limit = 3)

**find_all的扩展方法**

- <>.find()：搜索且只返回一个结果
- <>.find_parents()：在先辈节点中搜索，返回列表类型
- <>.find_parent()：在先辈节点中返回一个结果
- <>.find_next_siblings()：在后续平行节点中搜索，返回列表类型
- <>.find_next_sibling()：在后续平行节点中返回一个结果
- <>.find_previous_siblings()：在前序平行节点中搜索，返回列表类型
- <>.find_previous_sibling()：在前序平行节点中返回一个结果

区别：检索区域和返回结果不同

具体请参阅文档 https://beautifulsoup.readthedocs.io/zh_CN/latest/

**Exercise：**

1. 请提取页面中的股票代票、股票名称、公告日期这三项信息，并将数据存放在dataframe中。
2. 请提取出下述url页面中红色的数值。

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

## 使用一个网易的数据页面
url="http://quotes.money.163.com/data/caibao/yjgl_ALL.html?reportdate=20200930&sort=publishdate&order=desc&page=0"
     
def request_url(url):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36'
    headers = {'User-Agent': user_agent} 
    
    res = requests.get(url,headers=headers)
    res.encoding = 'utf-8'
    return res.text

#### 提取三项信息并将数据存入dataframe中

soup = BeautifulSoup(request_url(url), 'html.parser')




In [ ]:
#### 提取页面中红色的数值


3. 基于上述实例，尝试通过修改url = 'https://finance.sina.com.cn/roll/index.d.html?cid=56588&page=1'
中的关键词 page，实现自动翻页爬取多条信息，并存入本地数据库中。



**Exercise：**

1. 下面是一个批量下载百度图片的例子，请通过修改num，下载更多的图片，将num作为循环变量的参数写入程序中。
2. 当下载的图片很多时会发生各种错误，如何让程序稳定地运行？


In [ ]:
import requests
import re
import os
import time

pathname = 'D:\\图片\\'
if not os.path.exists(pathname):
    os.makedirs(pathname)
name = input('您要爬取的图片名称：')
num = 1
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}
url = 'https://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word='+name+'&pn='+str((num-1)*60)
res = requests.get(url,headers=headers)
# print(res.request.url)

html = res.content.decode() #将字节流转为unicode
addlist = re.findall('"objURL":"(.*?)",',html) #re.findall在字符串中找到正则表达式所匹配的所有子串，并返回一个列表
# print(len(addlist))
# print(addlist)

j=0
for i in addlist:
    j = j +1
    img = requests.get(i)
    f = open(pathname+name+str(j)+'.jpg','ab')
    print('---------正在下载第'+str(j)+'张图片----------')
    f.write(img.content)
    f.close()
print('下载完成')

**延申：**

1. 在之前的课程中，我们提到过另一种解析器lxml，它提供另一种方法XPath来提取网页中的数据，感兴趣的同学可参阅[click](https://www.cnblogs.com/cathycheng/p/11422685.html)

2. 思考如何爬取动态网页。

#### 小结：

1. 提取NavigableString的几种方法：text、strings、stripped_strings

2. 遍历网页标签树：上行，下行，平行

3. 利用find_all方法搜索标签树：对标签名称、属性、内容的搜索（正则表达式）
